# Introduction

```{include} ../README.md
:start-line: 6
:end-line: -4
```

:::{dropdown} LaminDB features

```{include} features-lamindb.md
```
:::

LaminHub is a data collaboration hub built on LaminDB similar to how GitHub is built on git.

:::{dropdown} LaminHub features

```{include} features-laminhub.md
```
:::

Basic features of LaminHub are free. Enterprise features, support, integration tests & wetlab plug-ins hosted in your or our infrastructure are available on a [paid plan](https://lamin.ai/pricing): please [reach out](https://lamin.ai/contact)!

## Quickstart

```{warning}

Public beta: Close to having converged a stable API, but some breaking changes might still occur.

```

### Setup LaminDB

1. Install the `lamindb` Python package:
    ```shell
    pip install 'lamindb[jupyter,bionty]'
    ```
2. [Sign up](https://lamin.ai/signup) for a free account (see more [info](https://lamin.ai/docs/setup)) and copy the API key.
3. Log in on the command line (data remains in your infrastructure, with Lamin having no access to it):
    ```shell
    lamin login <email> --key <API-key>
    ```

You can now init LaminDB instances like you init git repositories:

In [ ]:
!lamin init --schema bionty --storage ./lamin-intro  # or s3://my-bucket, gs://my-bucket as default storage

Because we passed `--schema bionty`, this instance mounted plug-in {mod}`lnschema_bionty`.

### Register a dataset

Track datasets using the {class}`~lamindb.Dataset` registry:

In [ ]:
import lamindb as ln
import pandas as pd

# track data flow in a notebook
ln.track()

# access a batch of data
df = pd.DataFrame(
    {"CD8": [1, 2, 3], "CD45": [3, 4, 5], "perturbation": ["DMSO", "IFNG", "DMSO"]},
    index=["measurement1", "measurement2", "measurement3"],
)

# create a dataset (versioning is optional)
dataset = ln.Dataset(df, name="Immune phenotyping", version="1")
# register dataset
dataset.save()

### Access a dataset

In [ ]:
# search a dataset
ln.Dataset.search("immune")

# query a dataset
dataset = ln.Dataset.filter(name__contains="phenotyping").first()

# view data flow
dataset.view_flow()

# describe metadata
dataset.describe()

# load the dataset
df = dataset.load()

### Define features & labels

Define features using the {class}`~lamindb.Feature` registry and labels using the {class}`~lamindb.ULabel` registry:

In [ ]:
# define features
names_types = [("CD8", "number"), ("CD45", "number"), ("perturbation", "category")]
features = [ln.Feature(name=name, type=type) for (name, type) in names_types]
ln.save(features)

# define perturbation labels
perturbations = [ln.ULabel(name=name) for name in df.perturbation.unique()]
ln.save(perturbations)

# define a parent label
is_perturbation = ln.ULabel(name="is_perturbation")
is_perturbation.save()
is_perturbation.children.add(*perturbations)

# view ontology
is_perturbation.view_parents(with_children=True)

### Validate & annotate data

Create a dataset with validated and linked features:

In [ ]:
# create dataset & validate features
dataset = ln.Dataset.from_df(df, name="Immune phenotyping 1")
# register dataset & link validated features
dataset.save()

(Warnings informed us that there are already a dataset and an underlying parquet file with matching hashes in storage; they are returned instead of duplicated.)

Annotate with perturbation labels:

In [ ]:
# an object to auto-complete features
features = ln.Feature.lookup()

# annotate with a label
dataset.labels.add(perturbations, feature=features.perturbation)

# show metadata
dataset.describe()

### Query for annotations

In [ ]:
# a look-up object for the ULabel registry
ulabels = ln.ULabel.lookup()

# all feature sets containing CD45
panels_with_cd45 = ln.FeatureSet.filter(features=features.cd45).all()

# query for all datasets measuring CD45
ln.Dataset.filter(feature_sets__in=panels_with_cd45).df()

# query for exactly one result measuring CD45 and being perturbed with IFNG
dataset = ln.Dataset.filter(
    feature_sets__in=panels_with_cd45, ulabels=ulabels.ifng
).one()

# permanently delete the dataset (without the permanent flag, moves to trash)
dataset.delete(permanent=True)

### Use biological types

Using the generic registries {class}`~lamindb.Feature` and {class}`~lamindb.ULabel` will get you pretty far.

But if you use an entity many times, you typically want a dedicated registry, which you can use to type your code & as an interface for public ontologies.

Let's do this for {class}`~lnschema_bionty.CellMarker` from plug-in {mod}`lnschema_bionty`:

In [ ]:
import lnschema_bionty as lb

# set a global organismm for multi-organism registries
lb.settings.organism = "human"

# create cell marker records from the public ontology
cell_markers = [lb.CellMarker.from_bionty(name=name) for name in ["CD8", "CD45"]]
ln.save(cell_markers)

## query CellMarker registry content as a DataFrame
lb.CellMarker.filter().df()

Because we now model cell markers explicitly, the features in our dataset have two types: data (the numerical expression measurements) and metadata (the categorical perturbation labels).

The simplest data structure in Python to account for different types of features is [AnnData](https://github.com/scverse/anndata). Let's use it to store our dataset:

In [ ]:
import anndata as ad

# stratify features by cell markers vs metadata
adata = ad.AnnData(df.loc[:, ["CD8", "CD45"]], obs=df.loc[:, ["perturbation"]])

# create dataset & validate features using the name field of CellMarker
dataset = ln.Dataset.from_anndata(
    adata, name="Immune phenotyping", field=lb.CellMarker.name
)
dataset.save()

# annotate with labels
dataset.labels.add(perturbations, feature=features.perturbation)

Query for a panel of cell markers & the linked datasets:

In [ ]:
# an object to auto-complete cell markers
cell_markers = lb.CellMarker.lookup()

# all cell marker panel containing CD45
panels_with_cd45 = ln.FeatureSet.filter(cell_markers=cell_markers.cd45).all()

# all datasets measuring CD45
ln.Dataset.filter(feature_sets__in=panels_with_cd45).df()

Use the {class}`~lnschema_bionty.ExperimentalFactor` & {class}`~lnschema_bionty.Tissue` ontologies to annotate the dataset with labels:

In [ ]:
# search the public ontology from the bionty store
lb.ExperimentalFactor.bionty().search("facs").head(2)

# create a record for facs
facs = lb.ExperimentalFactor.from_bionty(ontology_id="EFO:0009108")
facs.save()

# label with an inhouse assay
immune_assay1 = lb.ExperimentalFactor(name="Immune phenotyping assay 1")
immune_assay1.save()

# create a tissue from a public ontology
bone_marrow = lb.Tissue.from_bionty(name="bone marrow")
bone_marrow.save()

# annotate the dataset (no association with features)
dataset.experimental_factors.add(facs, immune_assay1)
dataset.tissues.add(bone_marrow)

# show metadata
dataset.describe()

### Append a new batch of data

In [ ]:
# assume we now run a pipeline in which we access a new batch of data
transform = ln.Transform(name=".fcs file ingestion", type="pipeline", version="1")
ln.track(transform)

# access a new batch of data with a different schema
df = pd.DataFrame(
    {
        "CD8": [2, 3, 3],
        "CD45": [3, 4, 5],
        "CD38": [4, 2, 3],
        "perturbation": ["IL2", "IL2", "IL2"],
        "concentration": [0.1, 0.2, 0.3],
    },
    index=["measurement4", "measurement5", "measurement6"],
)
adata = ad.AnnData(
    df.loc[:, ["CD8", "CD45", "CD38"]], obs=df.loc[:, ["perturbation", "concentration"]]
)

Because `"CD38"` and `"concentration"` are not yet part of the {class}`~lamindb.Feature` registry, they don't yet validate:

In [ ]:
file = ln.File.from_anndata(adata, description="Batch 2", field=lb.CellMarker.name)

Let's add them to the registries and re-create the file - now everything passes:

In [ ]:
lb.CellMarker.from_bionty(name="CD38").save()
ln.Feature(name="concentration", type="number").save()
file = ln.File.from_anndata(
    adata, description="Immune phenotyping batch 2", field=lb.CellMarker.name
)
file.save()

Create a new version of the dataset by linking both batches in a "sharded dataset":

In [ ]:
dataset_v2 = ln.Dataset([file, dataset.file], is_new_version_of=dataset)
dataset_v2.describe()
dataset_v2.save()
dataset_v2.view_flow()

You can load the entire dataset into memory as if it was one:

In [ ]:
dataset_v2.load()

Or iterate over its files:

In [ ]:
dataset_v2.files.df()

## More examples

### Understand data flow

View the sequence of data transformations ({class}`~lamindb.Transform`) in a project (from [here](docs:project-flow), based on [Schmidt _et al._, 2022](https://pubmed.ncbi.nlm.nih.gov/35113687/)):

```python
transform.view_parents()
```

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/b0geN1HDHXlORqMOOPay.svg" width="400">

Or, the generating flow of a file or dataset:

```python
file.view_flow()
```

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/KQmzmmLOeBN0C8Ykitjn.svg" width="800">


Both figures are based on mere calls to `ln.track()` in notebooks, pipelines & app.


### Manage biological registries

Create a cell type registry from public knowledge and add a new cell state (from [here](bio-registries)):

In [ ]:
import lnschema_bionty as lb

# create an ontology-coupled cell type record and save it
lb.CellType.from_bionty(name="neuron").save()

# create a record to track a new cell state
new_cell_state = lb.CellType(name="my neuron cell state", description="explains X")
new_cell_state.save()

# express that it's a neuron state
cell_types = lb.CellType.lookup()
new_cell_state.parents.add(cell_types.neuron)

In [ ]:
# view ontological hierarchy
new_cell_state.view_parents(distance=2)

### Leverage a mesh of instances

LaminDB is a distributed system like git. Similar to cloning a repository, collaborators can load your instance on the command-line using:

```shell
lamin load myhandle/myinstance
```

If you run `lamin save <notebook_path>`, you will save the notebook to your default storage location.

You can explore the notebook report corresponding to the quickstart [here](https://lamin.ai/laminlabs/lamindata/record/core/Transform?id=FPnfDtJz8qbEz8) in LaminHub.

### Manage custom schemas

LaminDB can be customized & extended with schema & app plug-ins building on the [Django](https://github.com/django/django) ecosystem. Examples are

- [lnschema_bionty](lnschema_bionty): Registries for basic biological entities, coupled to public ontologies.
- [lnschema_lamin1](https://github.com/laminlabs/lnschema-lamin1): Exemplary custom schema to manage samples, treatments, etc. 

If you'd like to create your own schema or app:

1. Create a git repository with registries similar to [lnschema_lamin1](https://github.com/laminlabs/lnschema-lamin1)
2. Create & deploy migrations via `lamin migrate create` and `lamin migrate deploy`

It's fastest if we do this for you based on our templates within an enterprise plan.

## Design

### Why?

We wrote a [blog post](https://lamin.ai/blog/2022/problems) about the key problems Lamin tries to solve when starting to work on it.

### Schema & API

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/XoTQFCmmj2uU4d2xyj9t.png" width="350px" style="background: transparent" align="right">

LaminDB provides a SQL schema for common entities: {class}`~lamindb.File`, {class}`~lamindb.Dataset`, {class}`~lamindb.Transform`, {class}`~lamindb.Feature`, {class}`~lamindb.ULabel` etc. - see the [API reference](reference) or the [source code](https://github.com/laminlabs/lnschema-core/blob/main/lnschema_core/models.py).

The core schema is extendable through plug ins (see blue vs. red entities in **graphic**), e.g., with basic biological ({class}`~lnschema_bionty.Gene`, {class}`~lnschema_bionty.Protein`, {class}`~lnschema_bionty.CellLine`, etc.) & operational entities (`Biosample`, `Techsample`, `Treatment`, etc.).

```{dropdown} What is the schema language?

Data models are defined in Python using the Django ORM. Django translates them to SQL tables.

[Django](https://github.com/django/django) is one of the most-used & highly-starred projects on GitHub (~1M dependents, ~73k stars) and has been robustly maintained for 15 years.

In the first year, LaminDB used SQLModel/SQLAlchemy -- we might bring back compatibility.

```

On top of the schema, LaminDB is a Python API that abstracts over storage & database access, data transformations, and (biological) ontologies.

The code for this is open-source & accessible through the dependencies & repositories listed below.
 
### Dependencies

- Data is stored in a platform-independent way: 
    - location → local, on AWS S3 or GCP Storage, accessed through `fsspec`
    - format → blob-like files or queryable formats like parquet, zarr, HDF5, TileDB, ...
- Metadata is stored in SQL: current backends are SQLite (small teams) and Postgres (any team size).
- Django ORM for schema management & metadata queries.
- Biological knowledge sources & ontologies: see [Bionty](https://lamin.ai/docs/bionty).

For more details, see the [pyproject.toml](https://github.com/laminlabs/lamindb/blob/main/pyproject.toml) file in lamindb & the linked repositories below.

### Repositories

LaminDB and its plug-ins consist in open-source Python libraries & publicly hosted metadata assets:

- [lamindb](https://github.com/laminlabs/lamindb): Core API, which builds on the [core schema](https://github.com/laminlabs/lnschema-core).
- [lnschema-bionty](https://github.com/laminlabs/lnschema-bionty): Registries for basic biological entities, coupled to public ontologies.
- [lnschema-lamin1](https://github.com/laminlabs/lnschema-lamin1): Exemplary custom schema to manage samples, treatments, etc.
- [lamindb-setup](https://github.com/laminlabs/lamindb-setup): Setup & configure LaminDB, client for LaminHub.
- [lamin-cli](https://github.com/laminlabs/lamin-cli): CLI for `lamindb` and `lamindb-setup`.
- [bionty](https://github.com/laminlabs/bionty): Accessor for public biological ontologies.
- [nbproject](https://github.com/laminlabs/nbproject): Metadata parser for Jupyter notebooks.
- [lamin-utils](https://github.com/laminlabs/lamin-utils): Generic utilities, e.g., a logger.
- [readfcs](https://github.com/laminlabs/readfcs): FCS file reader.
<!-- [bionty-assets](https://github.com/laminlabs/bionty-assets): Hosted assets of parsed public biological ontologies. -->

LaminHub is not open-sourced, and neither are plug-ins that model lab operations.


### Assumptions & principles

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/BunYmHkyFLITlM5MYQci.svg" width="350px" style="background: transparent" align="right">

1. Data is generated by instruments that process physical samples: it comes in batches stored as immutable files.
2. Files are transformed into more useful data representations, e.g.:
   - Summary statistics, e.g., count matrices for fastq files
   - Arrays of non-array-like input data (e.g., images)
   - Higher-level embeddings for lower-level array, text or graph representations
   - Concatenated arrays for large-scale atlas-like datasets
3. Semantics of high-level embeddings ("inflammatory", "lipophile") are anchored in experimental metadata and knowledge (ontologies)
4. Experimental metadata is another ontology type
5. Experiments measure features ({class}`~lamindb.Feature`, {class}`~lnschema_bionty.CellMarker`, ...)
6. Samples are annotated by labels ({class}`~lamindb.ULabel`, {class}`~lnschema_bionty.CellLine`, ...)
7. Learning and data warehousing both iterate transformations (see **graphic**, {class}`~lamindb.Transform`)
8. Basic biological entities should have the same meaning to anyone and across any data platform
9. Schema migrations should be easy

### Influences

LaminDB was influenced by many other projects, see {doc}`docs:influences`.

## Notebooks

- Find all tutorial & guide notebooks [here](https://github.com/laminlabs/lamindb/tree/main/docs/) and use cases [here](https://github.com/laminlabs/lamin-usecases).
- You can run these notebooks in hosted versions of JupyterLab, e.g., [Saturn Cloud](https://github.com/laminlabs/run-lamin-on-saturn), Google Vertex AI, Google Colab, and others.
- Jupyter Lab & Notebook offer a fully interactive experience, VS Code & others require using the CLI to track notebooks: `lamin track my-notebook.ipynb`

In [ ]:
!lamin delete --force lamin-intro